In [1]:
import gzip
import json

def fix_json_str(js):
    t = "[" + js
    t = t.replace("\n", ",")
    t = t[:-1]
    t += "]"
    return t

In [2]:
!pip install pydash

     |████████████████████████████████| 84 kB 5.7 MB/s             


In [3]:
def format_number(number):
    if number < 10:
        return "00" + str(number)
    elif number < 100:
        return "0" + str(number)
    else:
        return str(number)

In [4]:
import gc
gc.collect()

0

In [5]:
import json
import re
from pydash import get


alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

def get_first_sent(abst):
    return get(split_into_sentences(abst), 0)

In [6]:
import time
import pandas as pd

In [ ]:
data = []
errors = []
start=1400
end = 1600
for i in range(start,end):
    path = 'downloads/s2c/s2-corpus-{}.gz'.format(format_number(i))
    try:
        with gzip.open(path, 'r') as f:
            if (i % 20) == 0 and len(data):
                print("zip: ", i, " t ", time.time())

                df = pd.DataFrame(data)
                df["text"] = df["paperAbstract"].map(get_first_sent)
                df = df[["text", "s2Url", "doi", "journalName", "title", "fieldsOfStudy"]]
    #             df = df.drop(columns=["paperAbstract"])
                df.to_csv("downloads/s2c_{}_{}.csv".format(start, end), index = False, header=True)
            json_bytes = fix_json_str(f.read().decode('utf-8'))
            data += json.loads(json_bytes)
    except:
        print("ERROR:", path)
        errors += path
        

zip:  1420  t  1655465703.260025
zip:  1440  t  1655465974.528785
zip:  1460  t  1655466324.260637
zip:  1480  t  1655466760.2007763
zip:  1500  t  1655467277.3670113
zip:  1520  t  1655467885.0655344
zip:  1540  t  1655468577.9910586
zip:  1560  t  1655469359.7247002
zip:  1580  t  1655470184.2112591
ERROR: downloads/s2c/s2-corpus-1584.gz
ERROR: downloads/s2c/s2-corpus-1585.gz
ERROR: downloads/s2c/s2-corpus-1586.gz
ERROR: downloads/s2c/s2-corpus-1587.gz
ERROR: downloads/s2c/s2-corpus-1588.gz
ERROR: downloads/s2c/s2-corpus-1589.gz
ERROR: downloads/s2c/s2-corpus-1590.gz
ERROR: downloads/s2c/s2-corpus-1591.gz
ERROR: downloads/s2c/s2-corpus-1592.gz
ERROR: downloads/s2c/s2-corpus-1593.gz
ERROR: downloads/s2c/s2-corpus-1594.gz
ERROR: downloads/s2c/s2-corpus-1595.gz
ERROR: downloads/s2c/s2-corpus-1596.gz
ERROR: downloads/s2c/s2-corpus-1597.gz
ERROR: downloads/s2c/s2-corpus-1598.gz
ERROR: downloads/s2c/s2-corpus-1599.gz


In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv("downloads/s2c_0_300.csv",  lineterminator='\n')


In [4]:
len(df1)

6136274

In [6]:
df1.fieldsOfStudy.unique()

array(["['Computer Science']", "['Medicine']", '[]', ...,
       "['Psychology', 'Sociology', 'Engineering', 'Medicine']",
       "['History', 'Geography', 'Economics']",
       "['Computer Science', 'Mathematics', 'Medicine', 'Engineering']"],
      dtype=object)

In [9]:
!pip install pydash
from pydash import get

  Using cached pydash-5.1.0-py3-none-any.whl (84 kB)


In [10]:
df1["fields"] = df1.fieldsOfStudy.map(lambda a: get(eval(a), 0))

In [11]:
df1.fields.unique()

array(['Computer Science', 'Medicine', None, 'Art', 'Materials Science',
       'Engineering', 'Chemistry', 'Physics', 'Biology', 'History',
       'Psychology', 'Geography', 'Business', 'Mathematics', 'Geology',
       'Economics', 'Political Science', 'Environmental Science',
       'Philosophy', 'Sociology'], dtype=object)

In [12]:
df1

,text,s2Url,doi,journalName,title,fieldsOfStudy,fields
0,NaN,https://semanticscholar.org/paper/33b237709dbd...,10.14529/build190307,Bulletin of South Ural State University series...,USING PHOTO IMAGES TO DETERMINE THE GEOMETRIC ...,['Computer Science'],Computer Science
1,Automated medication dispensing systems for ho...,https://semanticscholar.org/paper/062e9c7579ad...,10.2190/W2BT-FGXQ-QL0G-YNL9,International Journal of Health Services,Clinical or Industrial Pharmacy? Case Studies ...,['Medicine'],Medicine
2,"In the future, the use of imaging methods to q...",https://semanticscholar.org/paper/771723e7e2d9...,10.1016/S1052-5149(02)00023-0,Neuroimaging clinics of North America,Noncoronary and coronary atherothrombotic plaq...,['Medicine'],Medicine
3,NaN,https://semanticscholar.org/paper/263f70167ba7...,NaN,NaN,국어 명사구 접속의 연구,[],None
4,NaN,https://semanticscholar.org/paper/9ea1148fa657...,NaN,NaN,Who'll Take My Place : When I'm Gone,['Art'],Art
...,...,...,...,...,...,...,...
6136269,NaN,https://semanticscholar.org/paper/4dc00bd1953b...,10.1111/J.1445-2197.1997.TB04602.X,The Australian and New Zealand journal of surgery,Where are we now with adjuvant therapy for rec...,['Medicine'],Medicine
6136270,"化粧品は各種界面活性剤や油剤等の混合系であり, 製品中からの界面活性剤の分析には, できるだ...",https://semanticscholar.org/paper/7315cf092574...,10.2116/BUNSEKIKAGAKU.51.921,Bunseki Kagaku,Determination of cationic and nonionic surfact...,['Chemistry'],Chemistry
6136271,NaN,https://semanticscholar.org/paper/0c6d7f8c8e3a...,NaN,NaN,Обретая Путь. Павел Флоренский в университетск...,[],None
6136272,NaN,https://semanticscholar.org/paper/d79f92d7fe00...,10.11501/3054420,NaN,二次イオン質量分析(SIMS)の機能拡張に関する基礎的研究,[],None


In [13]:
csvs = [
    "s2c_0_300.csv",
    "s2c_100_2M.csv",
    "s2c_180_340.csv",
    "s2c_340_500.csv",
    "s2c_1000_1300.csv",
    "s2c_1300_1500.csv",
    "s2c_1400_1600.csv",    
]

In [20]:
import time
df=df1
for csv in csvs:
    start_time = time.time()
    dfn= pd.read_csv("downloads/" + "s2c_0_300.csv", lineterminator='\n', error_bad_lines=False)
    df = df.append(dfn)
    print("--- %s seconds ---" % (time.time() - start_time))

--- 34.989346981048584 seconds ---
--- 40.790671825408936 seconds ---
--- 49.05041289329529 seconds ---
--- 57.82132124900818 seconds ---
--- 63.667720556259155 seconds ---
--- 69.1860704421997 seconds ---
--- 75.48966336250305 seconds ---


In [21]:
df

,text,s2Url,doi,journalName,title,fieldsOfStudy,fields
0,NaN,https://semanticscholar.org/paper/33b237709dbd...,10.14529/build190307,Bulletin of South Ural State University series...,USING PHOTO IMAGES TO DETERMINE THE GEOMETRIC ...,['Computer Science'],Computer Science
1,Automated medication dispensing systems for ho...,https://semanticscholar.org/paper/062e9c7579ad...,10.2190/W2BT-FGXQ-QL0G-YNL9,International Journal of Health Services,Clinical or Industrial Pharmacy? Case Studies ...,['Medicine'],Medicine
2,"In the future, the use of imaging methods to q...",https://semanticscholar.org/paper/771723e7e2d9...,10.1016/S1052-5149(02)00023-0,Neuroimaging clinics of North America,Noncoronary and coronary atherothrombotic plaq...,['Medicine'],Medicine
3,NaN,https://semanticscholar.org/paper/263f70167ba7...,NaN,NaN,국어 명사구 접속의 연구,[],None
4,NaN,https://semanticscholar.org/paper/9ea1148fa657...,NaN,NaN,Who'll Take My Place : When I'm Gone,['Art'],Art
...,...,...,...,...,...,...,...
6136269,NaN,https://semanticscholar.org/paper/4dc00bd1953b...,10.1111/J.1445-2197.1997.TB04602.X,The Australian and New Zealand journal of surgery,Where are we now with adjuvant therapy for rec...,['Medicine'],NaN
6136270,"化粧品は各種界面活性剤や油剤等の混合系であり, 製品中からの界面活性剤の分析には, できるだ...",https://semanticscholar.org/paper/7315cf092574...,10.2116/BUNSEKIKAGAKU.51.921,Bunseki Kagaku,Determination of cationic and nonionic surfact...,['Chemistry'],NaN
6136271,NaN,https://semanticscholar.org/paper/0c6d7f8c8e3a...,NaN,NaN,Обретая Путь. Павел Флоренский в университетск...,[],NaN
6136272,NaN,https://semanticscholar.org/paper/d79f92d7fe00...,10.11501/3054420,NaN,二次イオン質量分析(SIMS)の機能拡張に関する基礎的研究,[],NaN


In [24]:
dfb = df

In [ ]:
#df2 = pd.read_csv("downloads/s2c_180_340.csv", lineterminator='\n' )

#df= df1.append(df2)

#df.to_csv("downloads/s2c_11M_short.csv", index = False, header=True)

#dfs = pd.read_csv("downloads/s2c_11M_short.csv", lineterminator='\n', error_bad_lines=False)

In [22]:
!rm -rf cache_dir/

In [27]:
df = df[df.fieldsOfStudy.notna() & df.doi.notna()]



In [28]:
len(df)

26630816

In [30]:
df=df.dropna(subset=["text"])
#df=df.dropna(subset=["doi"])


In [32]:
len(df)

17779400

In [33]:
df = df[df["text"].str.len() > 25]
len(df)

17359784

In [35]:
df["fields"] = df.fieldsOfStudy.map(lambda a: get(eval(a), 0))

In [37]:
df.columns

Index(['text', 's2Url', 'doi', 'journalName', 'title', 'fieldsOfStudy',
       'fields'],
      dtype='object')

In [44]:
df

,text,s2Url,doi,journalName,title,fieldsOfStudy,fields
1,Automated medication dispensing systems for ho...,https://semanticscholar.org/paper/062e9c7579ad...,10.2190/W2BT-FGXQ-QL0G-YNL9,International Journal of Health Services,Clinical or Industrial Pharmacy? Case Studies ...,['Medicine'],Medicine
2,"In the future, the use of imaging methods to q...",https://semanticscholar.org/paper/771723e7e2d9...,10.1016/S1052-5149(02)00023-0,Neuroimaging clinics of North America,Noncoronary and coronary atherothrombotic plaq...,['Medicine'],Medicine
6,BackgroundCorneal endothelium (CE) images prov...,https://semanticscholar.org/paper/984774e366d3...,10.1186/s42490-019-0003-2,BMC Biomedical Engineering,Fully convolutional architecture vs sliding-wi...,"['Medicine', 'Computer Science']",Medicine
7,Falsified and substandard drugs are a global h...,https://semanticscholar.org/paper/0d0467c4cccb...,10.1371/journal.pone.0090601,PLoS ONE,Technologies for Detecting Falsified and Subst...,['Medicine'],Medicine
11,Objectives To explore the molecular epidemiolo...,https://semanticscholar.org/paper/878a852598fb...,10.1371/journal.pone.0211261,PLoS ONE,Mutation screening in non-syndromic hearing lo...,['Medicine'],Medicine
...,...,...,...,...,...,...,...
6136252,Computed tomography of the normal female pelvi...,https://semanticscholar.org/paper/5e0ae92aa402...,10.1177/028418519103200219,Acta Radiologica,Air in Vagina,['Medicine'],Medicine
6136254,Masih tingginya angka kematian ibu dan bayi me...,https://semanticscholar.org/paper/8e0d833dad81...,10.33653/JKP.V6I1.213,JURNAL KESEHATAN PERINTIS (Perintis's Health J...,FAKTOR-FAKTOR YANG MEMENGARUHI KEGAGALAN INISI...,['Physics'],Physics
6136256,Objective: To determine the earliest symptoms ...,https://semanticscholar.org/paper/4bc1e1639ab5...,10.1136/jnnp.2005.085373,"Journal of Neurology, Neurosurgery & Psychiatry",A diagnostic formulation for anosognosia in Al...,"['Psychology', 'Medicine']",Psychology
6136260,"Detailed measurements are made of the 79,81Br ...",https://semanticscholar.org/paper/9f9a62d79b31...,10.1002/PSSB.2221130138,Physica Status Solidi B-basic Solid State Physics,Temperature and Pressure Dependence of the Ele...,['Chemistry'],Chemistry


In [39]:
len(df)

17359784

In [40]:
df =df[df.fields.notna()]

In [48]:
df.to_csv("downloads/16M_semifiltered_all_domains.csv", index = False, header=True)

In [31]:
df = df[df.fields.isin(["Medicine", 'Environmental Science', 'Sociology', 'Psychology', 'Geology', 'Biology', 'Materials Science', 'Chemistry'])]

In [34]:
df.to_csv("downloads/s2c_5M_bioadjacent.csv",index = False, header=True)

In [29]:
!pip install pydash

  Using cached pydash-5.1.0-py3-none-any.whl (84 kB)


In [31]:
from pydash import get

In [60]:
df=df2

In [78]:
blacklist=["abstract", "discussion", "background", "summary", "rationale", "introduction", "publisher", "introduction", "statement", "purpose", "objective" ]
blacklist += [t + ":" for t in blacklist]
def remove_special_words(text, blacklist=blacklist ):
    words = text.split(" ")
    try:
        first=words[0]
    except:
        return text
    
    first = first.strip().lower()
#     if any(first in string for string in blacklist):
#         return " ".join(words[1:]).strip()
    for w in reversed(blacklist): #prioritize "Abstract:The" replace over "AbstractThe"
        if w in first:
            return " ".join([first.split(w)[1].capitalize()] + words[1:]).strip()
    return text

remove_special_words("ABSTRACT   The northern corn rootworm"), remove_special_words("purpose:The northern corn rootworm")

('The northern corn rootworm', 'The northern corn rootworm')

In [79]:
df["text"] = df["text"].map(remove_special_words)


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [81]:
dfs = df[["text", "doi", "fields"]]

In [89]:
dfs

,text,doi,fields
1,Automated medication dispensing systems for ho...,10.2190/W2BT-FGXQ-QL0G-YNL9,Medicine
2,"the future, the use of imaging methods to quan...",10.1016/S1052-5149(02)00023-0,Medicine
6,Corneal endothelium (CE) images provide valuab...,10.1186/s42490-019-0003-2,Medicine
7,Falsified and substandard drugs are a global h...,10.1371/journal.pone.0090601,Medicine
11,S To explore the molecular epidemiology of rar...,10.1371/journal.pone.0211261,Medicine
...,...,...,...
6136252,Computed tomography of the normal female pelvi...,10.1177/028418519103200219,Medicine
6136254,Masih tingginya angka kematian ibu dan bayi me...,10.33653/JKP.V6I1.213,Physics
6136256,To determine the earliest symptoms of anosogno...,10.1136/jnnp.2005.085373,Psychology
6136260,"Detailed measurements are made of the 79,81Br ...",10.1002/PSSB.2221130138,Chemistry


In [90]:
dfs.to_csv ("downloads/16M_small_alldomains_clean.csv", index = False, header=True)

In [63]:
dfx = df[["text", "s2Url", "doi", "journalName", "title", "fields", "fieldsOfStudy"]]

In [65]:
len(dfx)

1956472

In [64]:
dfx.to_csv ("downloads/s2c_100_2M.csv",index = False, header=True)

In [34]:
FoS = []
for d in data:
    FoS += d["fieldsOfStudy"]

print(set(FoS))

{'Business', 'Mathematics', 'Philosophy', 'Medicine', 'Environmental Science', 'Engineering', 'History', 'Art', 'Sociology', 'Psychology', 'Geology', 'Economics', 'Biology', 'Materials Science', 'Physics', 'Geography', 'Chemistry', 'Political Science', 'Computer Science'}
